### <b><span style='color:#F1C40F'>|</span> Home task</b>

- Choose any store from the initial dataset
- Check the presence of nans and fill them
- Make a forecast for 30, 180, 270, 365 days ahead
- Perform model evaluation

### <b><span style='color:#F1C40F'>|</span> References</b>


- Time Series at Scale [https://peerj.com/preprints/3190.pdf]
- Prophet Documentation [https://facebook.github.io/prophet/docs/quick_start.html#python-api]
- What is Time Series [https://medium.com/@ritusantra/what-is-time-series-and-components-of-time-series-c80b69ad5cb9]

### Choosing any store from the initial dataset

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

import plotly.express as px


def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    df.date = pd.to_datetime(df.date)
    df['day_of_week'] = df['date'].dt.day_name()
    return df


stores_df = pd.read_csv("train.csv")
stores_df = preprocess_data(stores_df)
stores_df

,date,store_nbr,family,sales,onpromotion,city,type_of_store,cluster,dcoilwtico,transactions,n_holidays,day_of_week
0,2013-01-01,1,Others,0.000,0,Quito,D,13,93.14,NaN,1.0,Tuesday
1,2013-01-01,1,Others,0.000,0,Quito,D,13,93.14,NaN,1.0,Tuesday
2,2013-01-01,1,Others,0.000,0,Quito,D,13,93.14,NaN,1.0,Tuesday
3,2013-01-01,1,BEVERAGES,0.000,0,Quito,D,13,93.14,NaN,1.0,Tuesday
4,2013-01-01,1,Others,0.000,0,Quito,D,13,93.14,NaN,1.0,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.133,0,Quito,B,6,47.57,2155.0,1.0,Tuesday
3000884,2017-08-15,9,Others,154.553,1,Quito,B,6,47.57,2155.0,1.0,Tuesday
3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,B,6,47.57,2155.0,1.0,Tuesday
3000886,2017-08-15,9,Others,121.000,8,Quito,B,6,47.57,2155.0,1.0,Tuesday


In [3]:
# shape of train set
print(f"Stores data shape - {stores_df.shape}")
# amount of unique stores
print(f'Amount of stores - {stores_df["store_nbr"].nunique()}')

Stores data shape - (3000888, 12)
Amount of stores - 54


In [4]:
# shape for 1 store
store_nbr = 1
store_data_shape = stores_df[stores_df["store_nbr"] == store_nbr].shape

print(f'Data shape for 1 store - {store_data_shape}')

Data shape for 1 store - (55572, 12)


In [5]:
# sum up sales for the day
def sum_sales_per_day(df: pd.DataFrame, store_number: int = 1) -> pd.DataFrame:
    day_level_df = df[df["store_nbr"] == store_number][
        ["date", "sales", "day_of_week"]
    ]\
        .groupby("date").agg(
        {
            "sales": "sum",
            "day_of_week": "first"
        }).reset_index()

    return day_level_df


day_level_df = sum_sales_per_day(stores_df)

In [6]:
# visualize sales
fig = px.line(day_level_df, x='date', y=["sales"], markers=True, title="Store sales")
fig.show()

In [7]:
# plot sales per each day of week
fig = px.box(day_level_df, x='day_of_week', y="sales", color="day_of_week",
             boxmode="overlay", points='all')
fig.update_layout(
    margin=dict(l=20, r=20, t=30, b=20),
    paper_bgcolor="LightSteelBlue",
    width=1400,
    height=700,
    title='Weekdays sales distribution',
)

In [8]:
from statsmodels.tsa.stattools import adfuller

adftest = adfuller(day_level_df[:30].set_index('date')['sales'].dropna()) #autolag = 'AIC', regression = 'n')
print("ADF Test Results")
print("Null Hypothesis: The series has an Unit Root")
print("P-Value:", adftest[1])
print("Note: If P-Value is smaller than 0.05, we reject the null Hypothesis and the series is Stationary")

ADF Test Results
Null Hypothesis: The series has an Unit Root
P-Value: 0.642869835836152
Note: If P-Value is smaller than 0.05, we reject the null Hypothesis and the series is Stationary


### Checking the presence of nans and filling them

In [9]:
# replace Zero values on NaN
day_level_df["sales"] = day_level_df["sales"].mask(
    day_level_df["sales"] == float(0), None)
day_level_df.head()

,date,sales,day_of_week
0,2013-01-01,NaN,Tuesday
1,2013-01-02,7417.148000,Wednesday
2,2013-01-03,5873.244001,Thursday
3,2013-01-04,5919.879001,Friday
4,2013-01-05,6318.785010,Saturday


In [10]:
# amount of Nan values
day_level_df["sales"].isna().sum()
print(f'NaN value counts - {day_level_df["sales"].isna().sum()}')

NaN value counts - 6


In [11]:
# load holidays event
event_df = pd.read_csv("holidays_events.csv")
event_df = preprocess_data(event_df)

event_df.head()

,date,type,locale,locale_name,description,transferred,day_of_week
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,Friday
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Sunday
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,Thursday
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,Saturday
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,Saturday


In [12]:
# merge sales data to look at the
event_df['date'] = pd.to_datetime(event_df['date'])

day_level_df[day_level_df['sales'].isna()].merge(
    event_df[["date", "description"]],
    how="left"
)

,date,sales,day_of_week,description
0,2013-01-01,NaN,Tuesday,Primer dia del ano
1,2014-01-01,NaN,Wednesday,Primer dia del ano
2,2015-01-01,NaN,Thursday,Primer dia del ano
3,2015-07-07,NaN,Tuesday,NaN
4,2016-01-01,NaN,Friday,Primer dia del ano
5,2017-01-01,NaN,Sunday,Primer dia del ano


In [13]:
# fill NaN with zero value
day_level_df.fillna(0).head()

,date,sales,day_of_week
0,2013-01-01,0.000000,Tuesday
1,2013-01-02,7417.148000,Wednesday
2,2013-01-03,5873.244001,Thursday
3,2013-01-04,5919.879001,Friday
4,2013-01-05,6318.785010,Saturday


# Making a forecast for 30, 180, 270, 365 days ahead and performing model evaluation for each forecast

In [14]:
# model evaluation
def evaluate_forecasting_model(actual_values:pd.Series, predicted_values:pd.Series, round_nbr:int=2) -> None:
    from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

    mape = mean_absolute_percentage_error(
        actual_values,
        predicted_values
    )
    mae = mean_absolute_error(
        actual_values,
        predicted_values
    )
    mse = mean_squared_error(
        actual_values,      
        predicted_values
    )

    print(f"MAE - {round(mae, round_nbr)}")
    print(f"MSE - {round(mse, round_nbr)}")
    print(f"MAPE - {round(mape, round_nbr)}")


In [15]:
from prophet import Prophet
from prophet.plot import plot_components_plotly

In [16]:
# preprocess data to needed format
fbp_set = day_level_df[['date', 'sales']]
fbp_set.rename(columns={"date": "ds", "sales": "y"}, inplace=True)
fbp_set.fillna(0, inplace=True)
fbp_set.head()

,ds,y
0,2013-01-01,0.000000
1,2013-01-02,7417.148000
2,2013-01-03,5873.244001
3,2013-01-04,5919.879001
4,2013-01-05,6318.785010


Adding event


In [17]:
# event_df = pd.read_csv("holidays_events.csv")
# event_df = preprocess_data(event_df)

# event_df.head()

In [18]:
# preprocess holidays dataframe
holiday_df = event_df.copy()
holiday_df.rename(
    columns={"date": "ds", "description": "holiday"}, inplace=True)
holiday_df = holiday_df[["ds", "holiday"]]
holiday_df.head()

,ds,holiday
0,2012-03-02,Fundacion de Manta
1,2012-04-01,Provincializacion de Cotopaxi
2,2012-04-12,Fundacion de Cuenca
3,2012-04-14,Cantonizacion de Libertad
4,2012-04-21,Cantonizacion de Riobamba


In [19]:
# split dataframe on train and test
window = 30
train, test = fbp_set[:-window], fbp_set[-window:]

In [20]:
# init and fit the model
m = Prophet(holidays=holiday_df)
m.fit(train)

18:09:42 - cmdstanpy - INFO - Chain [1] start processing
18:09:42 - cmdstanpy - INFO - Chain [1] done processing


In [21]:
# Create Future dates
future_sales = m.make_future_dataframe(periods=30)

# Predict sales
forecast = m.predict(future_sales)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

,ds,yhat,yhat_lower,yhat_upper
0,2013-01-01,-2766.583054,-4594.268827,-1076.494817
1,2013-01-02,8309.152430,6506.765656,10005.560724
2,2013-01-03,5751.586779,4051.538043,7590.712412
3,2013-01-04,7170.590328,5380.962227,8862.758497
4,2013-01-05,6309.086035,4639.812432,8165.621934


In [22]:
plot_components_plotly(m, forecast, figsize=(1000, 300))

In [23]:
# merge test set with forecasted values
benchmark_df = test.merge(forecast[["ds", "yhat"]], on="ds", how="left")

# Plot actual and forecasted data
fig = px.line(benchmark_df, x='ds', y=[
              "y", "yhat"], markers=True, title="Prophet forecast")
# Show plot
fig.show()

In [24]:
evaluate_forecasting_model(
    actual_values=benchmark_df['y'],
    predicted_values=benchmark_df['yhat'],
    round_nbr=3
)

MAE - 1042.733
MSE - 1623414.606
MAPE - 0.13


## Forecast for 30 days ahead

In [25]:
# split dataframe on train and test
window = 30
train, test = fbp_set[:-window], fbp_set[-window:]

# init and fit the model
m = Prophet(holidays=holiday_df)
m.fit(train)

# Create Future dates
future_sales = m.make_future_dataframe(periods=window)

# Predict sales
forecast = m.predict(future_sales)
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

# merge test set with forecasted values
benchmark_df = test.merge(forecast[["ds", "yhat"]], on="ds", how="left")

# Plot actual and forecasted data
fig = px.line(benchmark_df, x='ds', y=[
              "y", "yhat"], markers=True, title="Prophet forecast")
# Show plot
fig.show()

18:09:44 - cmdstanpy - INFO - Chain [1] start processing
18:09:45 - cmdstanpy - INFO - Chain [1] done processing


### Evaluation of the model in 30 days

In [26]:
evaluate_forecasting_model(
    actual_values=benchmark_df['y'],
    predicted_values=benchmark_df['yhat'],
    round_nbr=3
)

MAE - 1042.733
MSE - 1623414.606
MAPE - 0.13



## Forecast for 180 days ahead

In [27]:
# split dataframe on train and test
window = 180
train, test = fbp_set[:-window], fbp_set[-window:]

# init and fit the model
m = Prophet(holidays=holiday_df)
m.fit(train)

# Create Future dates
future_sales = m.make_future_dataframe(periods=window)

# Predict sales
forecast = m.predict(future_sales)
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

# merge test set with forecasted values
benchmark_df = test.merge(forecast[["ds", "yhat"]], on="ds", how="left")

# Plot actual and forecasted data
fig = px.line(benchmark_df, x='ds', y=[
              "y", "yhat"], markers=True, title="Prophet forecast")
# Show plot
fig.show()

18:09:46 - cmdstanpy - INFO - Chain [1] start processing
18:09:46 - cmdstanpy - INFO - Chain [1] done processing


### Evaluation of the model in 180 days

In [28]:
evaluate_forecasting_model(
    actual_values=benchmark_df['y'],
    predicted_values=benchmark_df['yhat'],
    round_nbr=3
)

MAE - 1093.268
MSE - 1946280.723
MAPE - 0.122


## Forecast for 270 days ahead

In [29]:
# split dataframe on train and test
window = 270
train, test = fbp_set[:-window], fbp_set[-window:]

# init and fit the model
m = Prophet(holidays=holiday_df)
m.fit(train)

# Create Future dates
future_sales = m.make_future_dataframe(periods=window+1)

# Predict sales
forecast = m.predict(future_sales)
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

# merge test set with forecasted values
benchmark_df = test.merge(forecast[["ds", "yhat"]], on="ds", how="left")

# Plot actual and forecasted data
fig = px.line(benchmark_df, x='ds', y=[
              "y", "yhat"], markers=True, title="Prophet forecast")
# Show plot
fig.show()

18:09:47 - cmdstanpy - INFO - Chain [1] start processing
18:09:47 - cmdstanpy - INFO - Chain [1] done processing


### Evaluation of the model in 270 days

In [30]:
evaluate_forecasting_model(
    actual_values=benchmark_df['y'],
    predicted_values=benchmark_df['yhat'],
    round_nbr=3
)

MAE - 1527.833
MSE - 3600590.187
MAPE - 4.291994646181718e+16


## Forecast for 360 days ahead

In [31]:
# split dataframe on train and test
window = 360
train, test = fbp_set[:-window], fbp_set[-window:]

# init and fit the model
m = Prophet(holidays=holiday_df)
m.fit(train)

# Create Future dates
future_sales = m.make_future_dataframe(periods=window+1)

# Predict sales
forecast = m.predict(future_sales)
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

# merge test set with forecasted values
benchmark_df = test.merge(forecast[["ds", "yhat"]], on="ds", how="left")

# Plot actual and forecasted data
fig = px.line(benchmark_df, x='ds', y=[
              "y", "yhat"], markers=True, title="Prophet forecast")
# Show plot
fig.show()

18:09:48 - cmdstanpy - INFO - Chain [1] start processing
18:09:48 - cmdstanpy - INFO - Chain [1] done processing


### Evaluation of the model in 360 days

In [32]:
evaluate_forecasting_model(
    actual_values=benchmark_df['y'],
    predicted_values=benchmark_df['yhat'],
    round_nbr=3
)

MAE - 1179.227
MSE - 2403854.088
MAPE - 1.8049461054335932e+16
